In [3]:
import webvtt
from os import listdir
from os.path import isfile, join
import pandas as pd
from pydub import AudioSegment
import os
import shutil
import natsort 
import wave
import xml.etree.ElementTree as ET

In [42]:
rootdir= "/home/megha/Desktop/b"

# 1) # make new directory and convert the wav file
#---------------------------------------------------------------------------------------------------------------
if True: #False  True
    i= 0
    for Dir in natsort.natsorted(os.listdir(rootdir)):
        if not os.path.exists(rootdir+"/"+Dir+"/temp"): 
            os.makedirs(rootdir+"/"+Dir+"/temp")
        i = i+1
        for file in natsort.natsorted(os.listdir(rootdir+"/"+Dir)):
            if(file.endswith(".WAV")):
                actual_filename = "folk_"+str(i)
                a = os.system('ffmpeg -i "{}" -acodec pcm_s16le -ac 1 -ar 16000 "{}/{}".wav'.format(rootdir+"/"+Dir+"/"+file, rootdir+"/"+Dir+"/temp", actual_filename))# (source,target(half path),name_of_file)
                print(a)
                if(a==256):
                        print("Did not get converted")
            if(file.endswith(".fln")):
                shutil.copyfile(rootdir+"/"+Dir+"/"+file, rootdir+"/"+Dir+"/temp/"+file)
            
#2) extract information from .fln file
#---------------------------------------------------------------------------------------------------------------
if True: #False  True
    for Dir in natsort.natsorted(os.listdir(rootdir)):        
        fileName=""
        #df  = pd.DataFrame(columns = ['A', 'B', 'C'])
        df1  = pd.DataFrame(columns = ['A', 'B', 'C'])
        df2  = pd.DataFrame(columns = ['A', 'B', 'C'])
        df3  = pd.DataFrame(columns = ['A', 'B', 'C'])
        for file in os.listdir(rootdir+"/"+Dir+"/temp"):            
            if(file.endswith(".wav")):
                fileName = file.split('.')[-2]+"_"
        for file in natsort.natsorted(os.listdir(rootdir+"/"+Dir+"/temp")):
            #print(file)
            if(file.endswith(".fln")):                
                if(file.endswith(".fln") and file.split('_')[-1]=="01.fln" ):
                    tree = ET.parse(rootdir+"/"+Dir+"/temp/"+file)
                    root = tree.getroot()
                    for contribution in root.findall("./contribution"):
                        children = contribution.getchildren()
                        time_array = []  
                        word = ""
                        for child in children:
                            if(child.tag == "time"):
                                time_array.append(child.attrib['time'])
                            if(child.tag == "uncertain"):
                                for c in child:
                                    word = word+" "+c.text
                            if(child.tag == "w"):
                                word = word+" "+child.text                        
                            df_a = pd.DataFrame([[time_array[0],time_array[-1],word]], columns=list('ABC'))
                        df1 = df1.append(df_a,ignore_index = True) 
                #print(df1.size)
                if(file.endswith(".fln") and file.split('_')[-1]=="02.fln" ): 
                    tree = ET.parse(rootdir+"/"+Dir+"/temp/"+file)
                    root = tree.getroot()
                    for contribution in root.findall("./contribution"):
                        children = contribution.getchildren()
                        time_array = []  
                        word = ""
                        for child in children:
                            if(child.tag == "time"):
                                time_array.append(child.attrib['time'])
                            if(child.tag == "uncertain"):
                                for c in child:
                                    word = word+" "+c.text
                            if(child.tag == "w"):
                                word = word+" "+child.text
                            df_b = pd.DataFrame([[time_array[0],time_array[-1],word]], columns=list('ABC'))
                        df2 = df2.append(df_b,ignore_index = True)
                #print(df2.size)
                if(file.endswith(".fln") and file.split('_')[-1]=="03.fln" ): 
                    tree = ET.parse(rootdir+"/"+Dir+"/temp/"+file)
                    root = tree.getroot()
                    for contribution in root.findall("./contribution"):
                        children = contribution.getchildren()
                        time_array = []  
                        word = ""
                        for child in children:
                            if(child.tag == "time"):
                                time_array.append(child.attrib['time'])
                            if(child.tag == "uncertain"):
                                for c in child:
                                    word = word+" "+c.text
                            if(child.tag == "w"):
                                word = word+" "+child.text
                            df_c = pd.DataFrame([[time_array[0],time_array[-1],word]], columns=list('ABC'))
                        df3 = df3.append(df_c,ignore_index = True)
                #print(df3.size) 
        frames = []
        frames = [df1, df2, df3]
        df = pd.concat(frames)
        df.reset_index(drop=True, inplace=True)
        check = df.iloc[::8, :].index
        time1 = []
        time2=  []
        starttime = []
        endtime = []
        string = []
        string_final = []
        for i in range(len(df)):
            x = df.get_values()[i]
            if(i in check):
                time1.append(float(x[0]))
                time2.append(float(x[1]))
                string.append(x[2])
            else:
                time2[-1] = float(x[1])
                string[-1] = string[-1] + x[2]
                if(i%8 == 7):
                    starttime.append(time1[-1])
                    endtime.append(time2[-1])
                    string_final.append(string[-1]) 
        string_final =  [x.upper() for x in string_final] 
        df_final = pd.DataFrame({'col1':starttime, 'col2':endtime, 'col3':string_final})
        df_final = df_final.drop(df_final[df_final.col3 ==""].index)         
        filename_list = []
        for i in range(0, len(df_final)):                    
            filename_list.extend([fileName+str(i)])    
        df_final['col4'] = filename_list
        print(df_final.shape)
        #----------------------------txt file creation-----------------------------------------------------------------   
        with open(rootdir+"/"+Dir+"/temp/"+"folk.trans.txt", "a") as file:
            for i in range(0, len(df_final)):
                file.write(df_final['col4'].iloc[i]+" "+df_final['col3'].iloc[i]+"\n")
        #----------------------------chuncking-----------------------------------------------------------------   
        for file1 in natsort.natsorted(os.listdir(rootdir+"/"+Dir+"/temp/")):
            if(file1.endswith(".wav")):
                myaudio = AudioSegment.from_file(rootdir+"/"+Dir+"/temp/"+file1, "wav")
                for i in range(0, len(df_final)):
                    START = (df_final['col1'].iloc[i] * 1000)
                    END = (df_final['col2'].iloc[i] * 1000)
                    chunk = myaudio[START:END]     
                    chunk_name = df_final['col4'].iloc[i]+".flac" 
                    if not os.path.exists(rootdir+"/"+Dir+"/temp/"+chunk_name):
                        print("chunking the audio", chunk_name)
                        chunk.export(rootdir+"/"+Dir+"/temp/"+chunk_name, format="wav")
            
#3) --------------------------------------cleaning folder---------------------------------------------
if False: # False True
    for Dir in natsort.natsorted(os.listdir(rootdir)):
        #print(Dir)
        for file in natsort.natsorted(os.listdir(rootdir+"/"+Dir)):
            print(file)
            if(file.endswith(".xml") or file.endswith(".WAV") or file.endswith(".fln") or file.endswith(".html")):
                os.remove(rootdir+"/"+Dir+"/"+file)
        for file in natsort.natsorted(os.listdir(rootdir+"/"+Dir+"/temp")):
            print(file)
            if(file.endswith(".fln") or file.endswith(".wav")):
                os.remove(rootdir+"/"+Dir+"/temp/"+file)
            else:
                shutil.move(rootdir+"/"+Dir+"/temp/"+file, rootdir+"/"+Dir+"/"+file)
        os.rmdir(rootdir+"/"+Dir+"/temp/")


0
(2, 4)
chunking the audio folk_1_0.flac
chunking the audio folk_1_1.flac


In [34]:
#df = df[['A','B', 'C']]
df.reset_index(drop=True, inplace=True)



In [44]:
pd.set_option('display.max_colwidth', -1)
df

,A,B,C
0,0.17,13.81,Herr Sch Sie sind in Veltheim an der Ohe geboren und haben dort Ihre Jugend verbracht Sie haben sicher noch allerlei Erinnerungen an alte Volksbräuche Können Sie uns darüber mal etwas erzählen
1,13.81,72.05,Ja ich bin in meiner Heimat Veltheim an der Ohe geboren und habe da auch meine Jugend zum Teil verlebt Wie ich denn älter wurde bin ich ja aus dem Dorf rausgegangen Aber sonst wie es so früher auf den Dörfern war es war noch nicht modern Ich kam auf 'ne Volksschule Da wurde denn unser Lehrer zum Teil noch mit Onkel angeredet und auch unsere Klasse die war manchmal achtzig Mann stark wohl achtzig Kinder stark Und wir kriegten dann erst eine kleine Schule Also wir hatten vormittags Schule und nachmittags Schule Und wie ich dann vielleicht acht Jahr kriegten wir eine neue Schule Na und denn ging das Dorfleben dann an Ja wie soll ich's nu
2,72.05,80.56,Ja wenn Sie nun in die junge Gesellschaft eintraten als Schulentlassener da hatten Sie doch dann manche neue Rechte und Pflichten
3,80.56,204.6,Ja ich kam denn in die Lehre Und früher war ja das also unter achtzehn Jahren wurde man ja noch immer von der sogenannten älteren Jugend als Kind betrachtet Wir durften uns das erste Jahr höchsten bis acht Uhr auf der Straße bewegen Auch von den Eltern wurden wir schon ziemlich durften nicht raus Und die größere Jugend die paßte dann schon auf daß wir ja pünktlich von der Straße waren Und wenn wir denn nicht gutwillig denn halfen sie mit irgend 'ner Peitsche oder versohlten uns e'ma 'n Hosenboden Also daß da eine gute Zucht drin war Und wenn man dann die Lehrzeit durch hatte man war denn Geselle geworden und auch die viel die Gesinde auf den Bauernhöfe wenn die ihre Lehrzeit beendet hatten daß sie als als vollwertig aufgenommen wurden dann war das sogenannte Fastnachtsvergnügen Da kam man dann in wurde man in diese Gesellschaft aufgenommen Hatte man sich aber irgendwie durch eine untadelige Tat sich gegen den Jugendgesetzen verstoßen also denn wurde man in diese Gesellschaft überhaupt nicht aufgenommen Und denn hatte das sogenannte Fastnachtsvergnügen denn kamen wir in einen Raum denn mußten wir uns erst einkaufen je nach dem Mittel 'n Taler und wer noch etwas besser konnte vier Mark und wer nu von kinderreichen Familien die beim Bauern waren die kriegten ja das ganze Jahr nur dreißig Taler da mußten die nu etwas mehr hatten
4,204.6,210.86,Und welche Bräuche wurden denn am Fasselabend nun ausgeführt
5,210.86,334.83,Ja erst e'mal wurden die sämtliche Jugend die im Dorf drin waren ach so die im Dorf drin waren die wurden dann alle zu einer war ja sogenanntes Komitee das wurden denn alle die wurden denn alle eingeladen Und dann wurden die einzelnen Posten verteilt jeder kriegte doch nu 'n Posten auch die Damen die noch untadelig waren Und die nur irgendwie schon sich 'n Baby angeschafft hatten und waren nicht verheiratet also von beiden Geschlechtern also die wurden auch von diesem Fest ausgeschlossen Die durften da nicht dran teilnehmen Na und denn wurde nachmittags sonntags nachmittags denn hatten lag eine große Liste aus und da waren sämtliche Damen aufgeführt Und das ging denn vom Alter und wurde den der Älteste der konnte anfangen und suchte sich dann seine Dame nach Wahl aus Und wir wo noch jünger waren wir mußten ja die nu nehmen die dann überblieben Also an'n Mann mußten sie alle gebracht werden Und dann kam die Musik und denn hatte so'n großer Bauer seinen Flur zur Verfügung gestellt und denn wurde mit Musik hingezogen und dann ging es wieder mit Jeder nahm seine Dame Auf dem Flur wirde wurde erst ein Rundtanz gemacht und dann zogen wir ganz durchs Dorf und zum Saal und denn gab es drei Ehrentänze die mußte man mit seiner Dame machen und dann war man ja frei Denn konnte man je nach Belieben wenn man Geschmack hatte mehr mit tanzen sonst konnte man sich 'ne Dame aussuchen die brauchte man nicht den ganzen Tag sich mit tanzen und auch beschäftigen
6,334.83,337.1,Wurden denn nicht vorher Würste gesammelt beim Umzieh

In [11]:
df.to_csv("/home/megha/Desktop/c/01.csv", sep='\t', encoding='utf-8')